<a href="https://colab.research.google.com/github/namakmurtaza/Twitter_Sentimental_Analysis/blob/main/Twitter_sentimenal_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from textblob import Word, TextBlob
from wordcloud import WordCloud
from nltk import word_tokenize

In [ ]:
pd.set_option("display.max_columns",None)
pd.set_option("display.width", 200)
pd.set_option("display.float_format", lambda x: "%.2f" % x)

In [ ]:
df_train = pd.read_csv("/content/twitter_training.csv")
df_valid = pd.read_csv("/content/twitter_validation.csv")

In [ ]:
df_train.head()

In [ ]:
df_train.columns

In [ ]:
df_train.columns=['id','information','status','comment']
df_train.head()

In [ ]:
df_valid.columns=['id','information','status','comment']
df_valid.head()

In [ ]:
df_train.info()

In [ ]:
df_valid.info()

In [ ]:
# Check for missing values
missing_values = df_train.isnull().sum()
print("Missing values per column:\n", missing_values)

In [ ]:
df_train = df_train.dropna(subset=['comment'])

In [ ]:
df_train.info()

In [ ]:
df_train["comment"] = df_train["comment"].str.lower()
df_valid["comment"] = df_valid["comment"].str.lower()

In [ ]:
df_train["comment"] = df_train["comment"].str.replace(r"[@&]\S+|&|http\S+|www\S+|[^\w\s]", "", regex=True).str.replace(r"\s+", " ", regex=True).str.strip()
df_valid["comment"] = df_valid["comment"].str.replace(r"[@&]\S+|&|http\S+|www\S+|[^\w\s]", "", regex=True).str.replace(r"\s+", " ", regex=True).str.strip()
df_train["comment"] = df_train["comment"].str.replace(r"[@&]\S+|&|http\S+|www\S+|\d+|[^\w\s]", "", regex=True) \
                                         .str.replace(r"\s+", " ", regex=True) \
                                         .str.strip()

df_valid["comment"] = df_valid["comment"].str.replace(r"[@&]\S+|&|http\S+|www\S+|\d+|[^\w\s]", "", regex=True) \
                                         .str.replace(r"\s+", " ", regex=True) \
                                         .str.strip()

In [ ]:
nltk.download("stopwords")
sw = stopwords.words("english")

In [ ]:
df_train["comment"] = df_train["comment"].apply(lambda x: " ".join(x for x in str(x).split() if x not in sw))
df_valid["comment"] = df_valid["comment"].apply(lambda x: " ".join(x for x in str(x).split() if x not in sw))

In [ ]:
temp_df_train = pd.Series(" ".join(df_train["comment"]).split()).value_counts()
temp_df_valid = pd.Series(" ".join(df_valid["comment"]).split()).value_counts()

In [ ]:
temp_df_train

In [ ]:
drop_train = temp_df_train[temp_df_train <= 1]
drop_valid = temp_df_valid[temp_df_valid <= 1]

In [ ]:
df_train["comment"] = df_train["comment"].apply(lambda x: " ".join(x for x in x.split() if x not in drop_train))
df_valid["comment"] = df_valid["comment"].apply(lambda x: " ".join(x for x in x.split() if x not in drop_valid))

In [ ]:
df_train.head(10)

In [ ]:
#Count information per category
plot1=df_train.groupby(by=["information","status"]).count().reset_index()
plot1.head()

In [ ]:
#Figure of comparison per branch
plt.figure(figsize=(20,6))
sns.barplot(data=plot1,x="information",y="id",hue="status")
plt.xticks(rotation=90)
plt.xlabel("Brand")
plt.ylabel("Number of tweets")
plt.grid()
plt.title("Distribution of tweets per Branch and Type");

In [ ]:
# Sentiment distribution
sns.countplot(x='status', data=df_train, palette='viridis')
plt.title("Distribution of Status")
plt.show()

In [ ]:
# Wordcloud
text = " ".join(i for i in df_train.comment)
wordcloud = WordCloud(background_color="white").generate(text)
plt.figure(figsize=(12,5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.title("Most Common Words in Tweets")
plt.axis("off")
plt.show()

In [ ]:
# Wordcloud positive
# "Positive" filter
positive_comments = df_train[df_train["status"] == "Positive"]["comment"]

text_positive = ''.join(i for i in positive_comments)
wordcloud = WordCloud(background_color="black").generate(text_positive)
plt.figure(figsize=(10,5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.title("Positive Common Words")
plt.axis("off")
plt.show()

In [ ]:
# Wordcloud negative
# "Negative" filter
negative_comments = df_train[df_train["status"] == "Negative"]["comment"]

text_positive = ''.join(i for i in negative_comments)
wordcloud = WordCloud(background_color="black").generate(text_positive)
plt.figure(figsize=(10,5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.title("Negative Common Words")
plt.axis("off")
plt.show()

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

In [ ]:
# Run these commands in separate cells
!pip install nltk
import nltk
nltk.download('punkt')
nltk.download('wordnet')

In [ ]:
# Method 1: Simple Split
tokens_text = [str(word).split() for word in df_train["comment"]]
tokens_counter = [item for sublist in tokens_text for item in sublist]
print("Number of tokens: ", len(set(tokens_counter)))

# Method 2: Regex Tokenization
import re
tokens_text = [re.findall(r'\w+', str(word)) for word in df_train["comment"]]
tokens_counter = [item for sublist in tokens_text for item in sublist]
print("Number of tokens: ", len(set(tokens_counter)))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import re

# Custom regex tokenization
def regex_tokenize(text):
    return re.findall(r'\w+', str(text))

# Convert text to numerical features
vectorizer = CountVectorizer(tokenizer=regex_tokenize, ngram_range=(1, 2))
X = vectorizer.fit_transform(df_train['comment'])
y = df_train['status']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a simple classifier
model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))